# import¶

In [24]:
# include working dir
!conda install -y gdown
!gdown --id 1UNsbV5xiYDq7vXnArOfAUDXIny08H_yi
!gdown --id 1VCbWE-DyVYZdDViH75ts62lrkr3rBe_V

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From (uriginal): https://drive.google.com/uc?id=1UNsbV5xiYDq7vXnArOfAUDXIny08H_yi
From (redirected): https://drive.google.com/uc?id=1UNsbV5xiYDq7vXnArOfAUDXIny08H_yi&confirm=t&uuid=b72046ba-2c40-41fe-a727-d903c3fe0a90
To: /kaggle/working/dataset_I_Train.zip
100%|█████████████████████████████████████████| 534M/534M [00:02<00:00, 231MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and wil

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.initializers import glorot_uniform
from keras.models import Model
from keras.layers import (Input, Conv1D, BatchNormalization, MaxPooling1D, 
                          AveragePooling1D, Flatten, Dense,Add, Activation, 
                          ZeroPadding1D, GlobalMaxPooling1D,MaxPool1D)
from keras.layers import Reshape, Permute, Concatenate, Dropout 
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,CSVLogger, EarlyStopping)
from sklearn.model_selection import KFold 
import tensorflow as tf
from tensorflow import keras

In [26]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

## load dataset

In [27]:
def train_test_split():
    # load dataset
    train_seq = pd.read_csv('/kaggle/working/dataset_I_Train.zip')
    test_seq = pd.read_csv('/kaggle/working/dataset_I_Test.zip')
    # split into input (X) and output (Y) variables
    X_train = train_seq.iloc[:,:-1]
    Y_train = train_seq[['output']]
    # encode class values as integers
    encoded_Y_train = pd.get_dummies(Y_train, ['output'])
    # Convert all columns to floats
    encoded_Y_train = encoded_Y_train.astype(float)
    X_test = test_seq.iloc[:,:-1]
    Y_test = test_seq[['output']] 
    encoded_Y_test = pd.get_dummies(Y_test, ['output'])
    # Convert all columns to floats
    encoded_Y_test = encoded_Y_test.astype(float)
    return X_train, encoded_Y_train, X_test, encoded_Y_test

In [28]:
# load dataset
X_train, y_train, X_test, y_test = train_test_split()

# Baseline Model

## CR-Former model Architectuer

In [29]:

def positional_encoding(seq_len, embedding_dim):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(embedding_dim)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(embedding_dim))
    angle_rads = pos * angle_rates

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate=0.1):
    # Input embedding layer
    input_layer = inputs
    x = input_layer
    
    # Positional Encoding
    seq_len = x.shape[1]
    embedding_dim = x.shape[2]
    pos_encoding = positional_encoding(seq_len, embedding_dim)
    x += pos_encoding[:, :seq_len, :]
    
    # Multi-Head Attention
    x = LayerNormalization()(x)
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    x = Add()([x, attn_output])
    
    # Feed Forward
    x = LayerNormalization()(x)
    ff_output = Dense(ff_dim, activation='relu')(x)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Dense(embedding_dim)(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    x = Add()([x, ff_output])
    
    return x


In [30]:
def identity_block(X, filters, stage, block):
    conv_name_base = f"res{stage}{block}_branch"
    F1, F2 = filters
    X_shortcut = X
    X = Conv1D(F1, 1, strides=1, padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv1D(F2, 1, strides=1, padding='valid', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = Add()([X, X_shortcut])
    return X

def convolutional_block(X, filters, stage, block, s=1):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2 = filters
    X_shortcut = X
    X = Conv1D(filters=F1, kernel_size=1, strides=s, padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv1D(filters=F2, kernel_size=1, strides=1, padding='valid', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X_shortcut = Conv1D(filters=F2, kernel_size=1, strides=s, padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X = Add()([X, X_shortcut])
    return X

def ResNet_Block(X, f_size, f_num, convblock, i1block, i2block):
    # stage 1-7
    for stage in range(1, 8):
        if stage%2==0:
            X = convolutional_block(X, filters=[f_num, f_num], stage=stage, block=convblock)
        else:
            X = convolutional_block(X, filters=[f_num, f_num], stage=stage, block=convblock)
            X = identity_block(X, [f_num, f_num], stage=stage, block=i1block)
            X = identity_block(X, [f_num, f_num], stage=stage, block=i2block)
    return X

def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate=0.1):
    x = LayerNormalization()(inputs)
    x = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(x, x)
    x = Dropout(dropout_rate)(x)
    x = Add()([inputs, x])
    x = LayerNormalization()(x)
    x = Dense(ff_dim, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(inputs.shape[-1])(x)
    x = Dropout(dropout_rate)(x)
    x = Add()([inputs, x])
    return x

def build_model(input_shape): 
    input_layer = Input(shape=input_shape)
    
    # Layer 1: Conv1D
    layer = Conv1D(filters=4, kernel_size=16, strides=1, padding='valid', kernel_initializer=glorot_uniform(seed=0))(input_layer)
    layer = BatchNormalization()(layer)

    # Layer 2: MaxPooling1D
    layer = MaxPooling1D(pool_size=2, strides=2)(layer)

    # Layer 3: Conv1D
    layer = Conv1D(filters=6, kernel_size=8, strides=1, padding='valid', kernel_initializer=glorot_uniform(seed=0))(layer)
    layer = BatchNormalization()(layer)

    # Layer 4: MaxPooling1D
    layer = MaxPooling1D(pool_size=2, strides=2)(layer)
    
    # For Skip Connection
    X_shortcut = layer
    
    # Layer 5: ResNet1
    layer = ResNet_Block(layer, 8, 8,'a','b','c')

    # Layer 6: ResNet2
    layer = ResNet_Block(layer, 16, 10,'d','e','f')
    
    # Reshaping And Padding For Additon And Skip Connection
    X_shortcut=Reshape([6,242])(X_shortcut)
    pad = ZeroPadding1D(padding=((4,0)))(X_shortcut)
    X_shortcut=Reshape([242,10])(pad)
    layer = Add()([layer, X_shortcut])

    # Layer 7: MaxPooling1D
    layer = MaxPooling1D(pool_size=2, strides=2)(layer)
    
    # For Skip Connection
    X_shortcut = layer

    # Layer 8: ResNet3
    layer = ResNet_Block(layer,16,12,'g','h','i') 

    # Reshaping And Padding For Additon And Skip Connection
    X_shortcut=Reshape([10,121])(X_shortcut)
    pad = ZeroPadding1D(padding=((2,0)))(X_shortcut)
    X_shortcut=Reshape([121,12])(pad)
    layer = Add()([layer, X_shortcut])

    # Layer 9: MaxPooling1D
    layer = MaxPooling1D(pool_size=2, strides=2)(layer)

    # For Skip Connection
    X_shortcut = layer

    # Layer 10: ResNet4
    layer = ResNet_Block(layer,4,14 ,'j','k','n')
    layer = MaxPooling1D(pool_size = 2, strides=2)(layer)   
    X_shortcut=Reshape([12,60])(X_shortcut)
    pad = ZeroPadding1D(padding=((2,0)))(X_shortcut)
    X_shortcut=Reshape([60,14])(pad)
    X_shortcut = MaxPooling1D(pool_size = 2, strides=2)(X_shortcut)
    layer = Add()([layer, X_shortcut])

    # Layer 11: AveragePooling1D
    layer = AveragePooling1D(pool_size=2, strides=1, padding='same')(layer)

    # Layer 12: Reshape
    #layer = Reshape((420,))(layer)

    # Layer 13-15: Transformer encoding
    num_heads = 5
    feed_forward_dim = 248
    for _ in range(3):
        layer = transformer_encoder(layer, num_heads, feed_forward_dim)
 
    # Layer 16: Fully connected
    layer = Flatten()(layer)
    layer = Dense(units=248, activation='relu')(layer)

    # Layer 17: Softmax
    output_layer = Dense(units=4, activation='softmax')(layer)

    model = Model(inputs=input_layer, outputs=output_layer, name='CRFormer')
    return model

## Train Model & Set parameter

In [34]:
# Optimization settings
num_samples = X_train.shape[1]
num_features = 1
lr = 2 * 10**-5
batch_size = 50
num_epochs = 100
opt = Adam(lr)

# define a callback functions
def getCallbacks():
    checkpoint = ModelCheckpoint(
        filepath='/kaggle/working/model.h5', # file path for saving the model
        monitor='val_loss', # quantity to monitor
        save_best_only=True, # save only the best model (based on the monitor quantity)
        save_weights_only=False, # save the entire model (not just the weights)
        mode='auto', # auto: the mode is inferred from the monitor quantity
        verbose=1 # print messages about the saving process
    )
    early_stopping = EarlyStopping(
        monitor='val_loss', # quantity to monitor
        patience=20, # number of epochs with no improvement before stopping
        verbose=1 # print messages about the early stopping process
    )
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', # quantity to monitor
        factor=0.2, # factor by which the learning rate will be reduced. new_lr = lr * factor
        patience=5, # number of epochs with no improvement before reducing learning rate
        verbose=1, # print messages about the learning rate reduction
        min_lr=1e-7 # lower bound on the learning rate
    )
    log_file_name = "/kaggle/working/log_fold_.csv"
    csv_logger = CSVLogger(log_file_name, separator=',', append=False)
    tensorboard = TensorBoard(
        log_dir='/kaggle/working/logs', # directory for storing the logs
        histogram_freq=1, # how often to compute activation and weight histograms (in epochs)
        write_graph=True, # whether to write the graph to TensorBoard
        write_images=True # whether to write image summaries to TensorBoard
    )
    return [checkpoint, early_stopping, csv_logger, reduce_lr, tensorboard]

model = build_model(input_shape=(num_samples, num_features))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [35]:
callbacks = getCallbacks()

# Fit the model on the training data
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size,
callbacks=callbacks, validation_data=(X_test, y_test))

Epoch 1/100
1450/1450 [==============================] - ETA: 0s - loss: 21423.2285 - accuracy: 0.6609
Epoch 1: val_loss improved from inf to 4960.26807, saving model to /kaggle/working/model.h5
1450/1450 [==============================] - 233s 84ms/step - loss: 21423.2285 - accuracy: 0.6609 - val_loss: 4960.2681 - val_accuracy: 0.7269 - lr: 2.0000e-05
Epoch 2/100
1450/1450 [==============================] - ETA: 0s - loss: 1934.3470 - accuracy: 0.7912
Epoch 2: val_loss improved from 4960.26807 to 2299.86475, saving model to /kaggle/working/model.h5
1450/1450 [==============================] - 122s 84ms/step - loss: 1934.3470 - accuracy: 0.7912 - val_loss: 2299.8647 - val_accuracy: 0.7750 - lr: 2.0000e-05
Epoch 3/100
1450/1450 [==============================] - ETA: 0s - loss: 836.0664 - accuracy: 0.8364
Epoch 3: val_loss improved from 2299.86475 to 1287.24158, saving model to /kaggle/working/model.h5
1450/1450 [==============================] - 118s 82ms/step - loss: 836.0664 - accura

## Visualize Model Training History

In [38]:
best_model = load_model('/kaggle/working/model.h5')
print(best_model.summary())

Model: "CRFormer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 1000, 1)]    0           []                               
                                                                                                  
 conv1d_8 (Conv1D)              (None, 985, 4)       68          ['input_5[0][0]']                
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 985, 4)      16          ['conv1d_8[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 max_pooling1d_24 (MaxPooling1D  (None, 492, 4)      0           ['batch_normalization_8[0]

## Save Model without_callbacks

In [39]:
model.save("my_model_without_callbacks.h5")
# best_model.save("my_model_without_callbacks.tflite")
# tensorflow_graph = tf.saved_model.load("/kaggle/working/my_model_without_callbacks.h5")
# config = model.get_config()  # Retrieve the config
# At loading time, register the custom objects with a `custom_object_scope`:
# custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation}
# with keras.utils.custom_object_scope(custom_objects):
#     new_model = keras.Model.from_config(config)
# serialize model to JSON
model_json = model.to_json()
with open("ECG_classification_model_interpatient_without_callbacks.json", "w") as json_file:
    json_file.write(model_json)
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('interpatient_model_without_callbacks.tflite', 'wb') as f:
    f.write(tflite_model)

print('TFLite Model save done!')

TFLite Model save done!


## Evaluate Model

In [41]:
best_model = load_model('/kaggle/working/model.h5')
accuracy_score(y_test, best_model.predict(X_test).round())

1797/1797 [==============================] - 38s 20ms/step


0.8518782608695652

## Communication using case study

In [42]:
# test_model
test_case = pd.read_csv("/kaggle/input/testdata/Book2.csv")
pred = best_model.predict(test_case).round()
out_col = ['output_CAD', 'output_CHF', 'output_MI', 'output_Normal']
output = {out_col[i]:pred[0][i] for i in range(4)}
output_class = max(output, key=output.get).split('_')[-1]
"output_class = " + output_class

1/1 [==============================] - 0s 87ms/step


'output_class = Normal'

# End Model Training Notebook